In [1]:
import os
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns

from dataset.OnlineDataset import OnlineDataset
import torch

In [2]:
customer=[13,35,74,82,87]

In [ ]:
ds = OnlineDataset(raw_data_path='../data/1_processed',
                   sliding_window_size=336,
                   sliding_window_offset=336,
                   forecast_size=0,
                   customer=87,
                   mode='test',
                   device='cpu')
battery_capacity = ds.getBatteryCapacity()
max_power = ds.getBatteryCapacity()/4
print(f'Battery capacity: {battery_capacity} kWh, Max power: {max_power} kW')

Battery capacity: 6.0 kWh, Max power: 1.5 kW


# WO Battery

In [17]:
cost = 0
curr_ds = ds[0]
prosumption = curr_ds['prosumption']
price = curr_ds['price']
for j in range(336):

        cost  += prosumption[j] * price[j] if prosumption[j]>=0 else prosumption[j]*0.1
cost

tensor(10.7644)

# Gurobi Optimizer

In [6]:
final_cost = 0.0
# for i in range(51):
curr_ds = ds[0]
prosumption = curr_ds['prosumption']
price = curr_ds['price']
T=336
max_charge = 0.25
max_discharge = -0.25
battery_capacity = 2.0
big_M = 1e4  # A sufficiently large number

# Create model
model = gp.Model("BatteryCharging")
model.Params.OutputFlag = 1

# Variables
action = model.addVars(T, lb=max_discharge, ub=max_charge, name="action")
soe = model.addVars(T + 1, lb=0.0, ub=battery_capacity, name="soe")
z = model.addVars(T, lb=0.0, ub=10, vtype=GRB.CONTINUOUS, name='actual_price')
b = model.addVars(T, vtype=GRB.BINARY, name="b")

# Initial SoE
model.addConstr(soe[0] == 0.0, name="initial_soe")

eps = 0.0001
M = 20 + eps

for t in range(T):
    # Battery dynamics
    model.addConstr(soe[t + 1] == soe[t] + action[t], name=f"soe_update_{t}")
    model.addConstr(action[t]+prosumption[t] >= 0 + eps - M * (1 - b[t]), name="bigM_constr1")
    model.addConstr(action[t]+prosumption[t] <= 0 + M * b[t], name="bigM_constr2")

    # Add indicator constraints
    model.addConstr((b[t] == 1) >> (z[t] == price[t]), name="indicator_constr1")
    model.addConstr((b[t] == 0) >> (z[t] == 0.1), name="indicator_constr2")



# Objective function: minimize cost
obj = gp.quicksum(
    (prosumption[t] + action[t]) * z[t]
    for t in range(T)
)
model.setObjective(obj, GRB.MINIMIZE)

# Optimize
model.optimize()

if model.status == GRB.OPTIMAL:
    actions = [action[t].X for t in range(T)]
    soes = [soe[t].X for t in range(T+1)]
    used_price = [z[t].X for t in range(T)]
    for t in range(T):
        curr_price = price[t] if (prosumption[t]+actions[t]) >= 0 else 0.1
        final_cost += (prosumption[t]+actions[t])*curr_price

print(f'final cost: {final_cost}')

Set parameter Username
Set parameter LicenseID to value 2708918


GurobiError: HostID mismatch (licensed to 5d2360b0, hostid is 5d3e0573)